![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F08+-+R%2Fdepricated&file=08Tools+-+GCS+FUSE.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08Tools%20-%20GCS%20FUSE.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F08%2520-%2520R%2Fdepricated%2F08Tools%2520-%2520GCS%2520FUSE.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08Tools%20-%20GCS%20FUSE.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/08%20-%20R/depricated/08Tools%20-%20GCS%20FUSE.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 08Tools - GCS FUSE

**IN ACTIVE DEVELOPMENT - NOT COMPLETE**

Dev Notes:
- R Kernel:
    - use notebook instance with R kernel
    - or, install R here with needed libraries
- Demonstrate BQ > GCS > R with GCS FUSE Mount

Workflow:
- Initiate export of data from BigQuery to GCS Bucket
- Read data from GCS into R dataframe from GCS FUSE mount location
- Delete data from GCS Bucket
    
**Resources:**
- Vertex AI Training Jobs, read and write Cloud Storage file with Cloud Storage FUSE [link](https://cloud.google.com/vertex-ai/docs/training/code-requirements#fuse)
- Blog on Mounting Cloud Storage Bucket in Vertex AI Workbench [link](https://cloud.google.com/blog/topics/developers-practitioners/cloud-storage-file-system-vertex-ai-workbench-notebooks)
- [Cloud Storage FUSE](https://cloud.google.com/storage/docs/gcs-fuse)
    
**Prerequisites:**
- [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)

---
## Preparation

When using a Vertex AI Workbench the instance already has GCS Fuse installed.  Use the following steps at a terminal to mount a local folder for use:

```
cd ~/
mkdir -p gcs
gcsfuse --implicit-dirs --rename-dir-limit=100 --disable-http2 --max-conns-per-host=100 "/home/jupyter/gcs"
```

When running a Vertex AI Training job this mount is already done for you at the `/gcs/` location.

---
## Setup

inputs:

In [1]:
project_id <- system('gcloud config get-value project', intern = TRUE)
project_id

[1] "statmike-mlops-349915"

In [17]:
series = '08'
experiment = 'gcs_fuse'

bq_project <- project_id
bq_dataset <- 'fraud'
bq_table <- 'fraud_prepped'

var_target <- 'Class'
var_omit <- 'transaction_id'

packages:

In [75]:
library(bigrquery)
library(data.table)


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




---
## Export Data From BigQuery to GCS

Use a SQL Query to initiate and export from BigQuery to GCS at a specified location.  This example show exporting to `.csv` files with a wildcard.  The example data is small enough that is outputs to a single file but this code would work with much larger datasets that export to multiple `.csv` files as well.

**Resources:**

- [BigQuery data exports](https://cloud.google.com/bigquery/docs/exporting-data#sql)


### Construct the query string:

In [24]:
query <- "
EXPORT DATA
    OPTIONS (
        uri = 'gs://%s/%s/%s/data/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT * EXCEPT(%s, splits)
    FROM `%s.%s.%s`
    WHERE splits = 'TRAIN'
)"

In [25]:
cat(query)


EXPORT DATA
    OPTIONS (
        uri = 'gs://%s/%s/%s/data/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT * EXCEPT(%s, splits)
    FROM `%s.%s.%s`
    WHERE splits = 'TRAIN'
)

In [26]:
query <- sprintf(query, project_id, series, experiment, var_omit, bq_project, bq_dataset, bq_table)

In [27]:
cat(query)


EXPORT DATA
    OPTIONS (
        uri = 'gs://statmike-mlops-349915/08/gcs_fuse/data/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT * EXCEPT(transaction_id, splits)
    FROM `statmike-mlops-349915.fraud.fraud_prepped`
    WHERE splits = 'TRAIN'
)

### Run the Query (using library bigrquery)

In [32]:
export <- bq_perform_query(query, billing = bq_project)

In [33]:
bq_job_wait(export)

### List the Exported Files:

In [77]:
files <- list.files(
    path = sprintf('~/gcs/%s/%s/%s/data', project_id, series, experiment),
    pattern = '*.csv',
    full.names = TRUE
)
files

[1] "/home/jupyter/gcs/statmike-mlops-349915/08/gcs_fuse/data/000000000000.csv"

---
## Read the Data Into An R Dataframe

The code here will read all `.csv` files at the bucket location into a single R dataframe.

In [78]:
path <- sprintf('~/gcs/%s/%s/%s/data', project_id, series, experiment)
df <- 
    files %>%
    map_df(~fread(.))

---
## Use The Data To Create A Model

In [80]:
model <- glm(
    Class ~ .,
    data = df,
    family = binomial)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [81]:
summary(model)


Call:
glm(formula = Class ~ ., family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.9067  -0.0297  -0.0196  -0.0126   4.6681  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -8.364e+00  2.708e-01 -30.893  < 2e-16 ***
Time        -3.794e-06  2.479e-06  -1.530 0.125941    
V1           1.244e-01  4.665e-02   2.666 0.007667 ** 
V2          -4.491e-02  6.020e-02  -0.746 0.455685    
V3          -1.534e-02  5.767e-02  -0.266 0.790218    
V4           6.668e-01  7.603e-02   8.770  < 2e-16 ***
V5           1.355e-01  6.869e-02   1.973 0.048496 *  
V6          -8.460e-02  7.614e-02  -1.111 0.266556    
V7          -1.222e-01  6.931e-02  -1.763 0.077897 .  
V8          -1.758e-01  3.225e-02  -5.452 4.98e-08 ***
V9          -4.114e-01  1.162e-01  -3.541 0.000398 ***
V10         -8.457e-01  9.909e-02  -8.535  < 2e-16 ***
V11         -1.359e-01  9.153e-02  -1.485 0.137524    
V12          8.785e-02  9.681e-02   0.9

---
## Delete The Exported Data

Optionally, delete the data exported from BigQuery to GCS.  Depending on your workflow you may want to adopt the practice of cleaning up the exported data with each run of this code when it is finished.

In [82]:
sapply(files, unlink)

/home/jupyter/gcs/statmike-mlops-349915/08/gcs_fuse/data/000000000000.csv 
                                                                        0